<a href="https://colab.research.google.com/github/pourmand1376/Polyp_detection/blob/main/KUMC_First_RUN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download Dataset and preprocess that

In [ ]:
%%bash
gdown 1HPx_GYIl0ZwDxhBUwUiGLkR0T41LS2mj
mkdir KUMC
tar xzf PolypsSet_KUMC.tar.gz --directory KUMC

Downloading...
From: https://drive.google.com/uc?id=1HPx_GYIl0ZwDxhBUwUiGLkR0T41LS2mj
To: /content/PolypsSet_KUMC.tar.gz
100%|██████████| 2.47G/2.47G [00:16<00:00, 145MB/s]


In [ ]:
from os import walk, path, rename
def rename_files_in_folder(folder_path):
    """
    rename items in subfolders to main folders like 1/1.txt to 1_1.txt
    """
    for dirpath, _, files in walk(folder_path):
        for f in files:
            rename(path.join(dirpath, f), path.join(dirpath, path.split(dirpath)[-1] + '_' + f))

In [ ]:
rename_files_in_folder('/content/KUMC/test2019/Annotation')
rename_files_in_folder('/content/KUMC/test2019/Image')
rename_files_in_folder('/content/KUMC/val2019/Annotation')
rename_files_in_folder('/content/KUMC/val2019/Image')

In [ ]:
%%bash
# recursively move files from subfolders to folder
cd /content/KUMC/val2019/Image
mv */* .
cd /content/KUMC/val2019/Annotation
mv */* .
cd /content/KUMC/test2019/Image
mv */* .
cd /content/KUMC/test2019/Annotation
mv */* .

In [ ]:
#remove empty folders 
!find . -type d -empty -delete

In [ ]:
%%bash
cd /content/KUMC/test2019
mv Image/ images
cd /content/KUMC/train2019
mv Image/ images
cd /content/KUMC/val2019
mv Image/ images

In [ ]:
!zip -r KUMC.zip /content/KUMC

Streaming output truncated to the last 5000 lines.
  adding: content/KUMC/train2019/images/8140.jpg (deflated 6%)
  adding: content/KUMC/train2019/images/15586.jpg (deflated 2%)
  adding: content/KUMC/train2019/images/25747.jpg (deflated 2%)
  adding: content/KUMC/train2019/images/1727.jpg (deflated 2%)
  adding: content/KUMC/train2019/images/18606.jpg (deflated 1%)
  adding: content/KUMC/train2019/images/18616.jpg (deflated 1%)
  adding: content/KUMC/train2019/images/10134.jpg (deflated 4%)
  adding: content/KUMC/train2019/images/6848.jpg (deflated 8%)
  adding: content/KUMC/train2019/images/12379.jpg (deflated 7%)
  adding: content/KUMC/train2019/images/28752.jpg (deflated 15%)
  adding: content/KUMC/train2019/images/2950.jpg (deflated 2%)
  adding: content/KUMC/train2019/images/3421.jpg (deflated 2%)
  adding: content/KUMC/train2019/images/24048.jpg (deflated 1%)
  adding: content/KUMC/train2019/images/20609.jpg (deflated 2%)
  adding: content/KUMC/train2019/images/12534.jpg (deflat

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
!cp /content/KUMC.zip /content/drive/MyDrive/KUMC.zip

In [ ]:
!ls -lh

total 4.7G
drwxr-xr-x 5 root root 4.0K May  2 13:36 KUMC
-rw-r--r-- 1 root root 2.4G May  2 14:38 KUMC.zip
-rw-r--r-- 1 root root 2.3G May  2 13:34 PolypsSet_KUMC.tar.gz
drwxr-xr-x 1 root root 4.0K Apr 29 03:19 sample_data
drwxr-xr-x 9 root root 4.0K May  2 13:37 yolov5
-rw------- 1 root root  15M May  2 13:38 yolov5s.pt


## Resize PASCAL VOC (Currently not in use)

In [ ]:
# make files good for resize
for item in ['test2019','train2019','val2019']:
    !mv /content/KUMC/{item}/images/*.jpg /content/KUMC/{item}
    !mv /content/KUMC/{item}/Annotation/*.xml /content/KUMC/{item}

In [ ]:
!git clone https://github.com/italojs/resize_dataset_pascalvoc /content/resize  

Cloning into '/content/resize'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 50 (delta 14), reused 11 (delta 11), pack-reused 29
Unpacking objects: 100% (50/50), done.


In [ ]:
for item in ['test2019','train2019','val2019']: # do resize
    !python /content/resize/main.py -p /content/KUMC/{item} --output /content/KUMC/{item}_resized --new_x 640 --new_y 480 #--save_box_images 1

In [ ]:
for item in ['test2019','train2019','val2019']: #get back at default directory strcuture
    !rm -rf /content/KUMC/{item}
    !mkdir -p /content/KUMC/{item}/images
    !mkdir -p /content/KUMC/{item}/Annotation
    !mv /content/KUMC/{item}_resized/*.jpg /content/KUMC/{item}/images/ 
    !mv /content/KUMC/{item}_resized/*.xml /content/KUMC/{item}/Annotation

## Convert to YOLO Format

In [ ]:
pip install pylabel

In [ ]:
from pylabel import importer
dataset = importer.ImportVOC(path="/content/KUMC/train2019/Annotation", path_to_images="/content/KUMC/train2019/images")
dataset.export.ExportToYoloV5(output_path='/content/KUMC/train2019/labels',
    yaml_file='dataset.yaml', cat_id_index=0) 

In [ ]:
dataset = importer.ImportVOC(path="/content/KUMC/test2019/Annotation", path_to_images="/content/KUMC/tet2019/images")
dataset.export.ExportToYoloV5(output_path='/content/KUMC/test2019/labels',
    yaml_file='dataset.yaml', cat_id_index=0) 

In [ ]:
dataset = importer.ImportVOC(path="/content/KUMC/val2019/Annotation", path_to_images="/content/KUMC/val2019/images")
dataset.export.ExportToYoloV5(output_path='/content/KUMC/val2019/labels',
    yaml_file='dataset.yaml', cat_id_index=0) 

In [ ]:
%%bash
cd /content/KUMC/
echo "
train: /content/KUMC/train2019/images
val: /content/KUMC/val2019/images
test: /content/KUMC/test2019/images
nc: 2
names: [hyperplastic, adenomatous]
" > database.yaml

## Use YOLO

In [ ]:
%%bash
git clone https://github.com/ultralytics/yolov5  # clone
cd yolov5
pip install -r requirements.txt  # install

In [ ]:
!python /content/yolov5/train.py --img 640 --batch 64 --epochs 3 --data /content/KUMC/database.yaml --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=, data=/content/KUMC/database.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=3, batch_size=64, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-169-g1994ad9 torch 1.11.0+cu113 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv